In [1]:
from torch import nn, optim
import torch
import numpy as np
from torch.utils import data
from module.flow import cnf
from math import log, pi
import os
from tqdm import tqdm

import random
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as dset
import argparse
from os import path
from numpy.random import uniform, randn
from sklearn.model_selection import train_test_split


In [2]:
def standard_normal_logprob(z):
    dim = z.size(-1)
    log_z = -0.5 * dim * log(2 * pi)
    return log_z - z.pow(2) / 2

### Dataset

In [5]:
class MyDataset(Dataset):
    def __init__(self, latents, attributes, transform=None):
        self.latents = latents
        self.attributes = attributes
        self.transform = transform

    def __getitem__(self, index):
        x = self.latents[index]
        y = self.attributes[index]


        if self.transform:
            x = self.transform(x)
            y = self.transform(y)

        return x, y

    def __len__(self):
        return len(self.latents)

# TEST???


In [8]:
torch.manual_seed(0)


In [15]:
flow_modules = '8-8-8-8-8'
cond_size = 17
num_blocks = 1
inputDim = 1
prior = cnf(inputDim, flow_modules, cond_size, num_blocks)

Training T : 1.0
Number of trainable parameters of Point CNF: 1835


In [16]:
prior

SequentialFlow(
  (chain): ModuleList(
    (0): MovingBatchNorm1d(1, eps=0.0001, decay=0.1, bn_lag=0, affine=True)
    (1): CNF(
      (odefunc): ODEfunc(
        (diffeq): ODEnet(
          (layers): ModuleList(
            (0): ConcatSquashLinear(
              (_layer): Linear(in_features=1, out_features=8, bias=True)
              (_hyper_bias): Linear(in_features=18, out_features=8, bias=False)
              (_hyper_gate): Linear(in_features=18, out_features=8, bias=True)
            )
            (1): ConcatSquashLinear(
              (_layer): Linear(in_features=8, out_features=8, bias=True)
              (_hyper_bias): Linear(in_features=18, out_features=8, bias=False)
              (_hyper_gate): Linear(in_features=18, out_features=8, bias=True)
            )
            (2): ConcatSquashLinear(
              (_layer): Linear(in_features=8, out_features=8, bias=True)
              (_hyper_bias): Linear(in_features=18, out_features=8, bias=False)
              (_hyper_gate): Li

In [6]:
datas = np.load("./1d_data/my_1d_data.npy")

In [7]:
datas[:10]


array([[-1.0747095 , -0.16771168, -0.12928766, -0.18957202],
       [ 0.12670742,  0.05116083,  0.9276154 ,  1.6304451 ],
       [ 0.90425605,  1.0532261 ,  0.88205355,  1.5947601 ],
       [ 1.0807073 ,  1.4631921 , -0.1918909 , -0.38346523],
       [ 0.9047382 ,  1.4226955 , -0.07398555, -0.24800563],
       [ 0.93705505,  1.8759551 ,  1.0437909 ,  1.8301312 ],
       [-0.01835815, -0.637534  , -0.1093017 , -0.438529  ],
       [ 1.255879  ,  1.2127938 ,  0.02129816, -0.39670995],
       [ 1.0918123 ,  1.429814  ,  1.2885988 ,  1.1864866 ],
       [-1.2372917 , -0.7683743 , -1.2420946 , -0.7944485 ]],
      dtype=float32)

In [8]:
datas.shape

(501, 4)

In [22]:
def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.isdir(path):
            os.makedirs(path, mode=0o777)

In [28]:
def load_my_1d(base_dir):
    if not path.exists(base_dir + '/my_1d_data/'):
        mkdir(base_dir + '/my_1d_data/')

        def gen_my_1d(hetero=False):

            np.random.seed(0)
            Npoints = 1002
            x0 = uniform(-1, 0, size=int(Npoints / 3))
            x1 = uniform(1.7, 2.5, size=int(Npoints / 3))
            x2 = uniform(4, 5, size=int(Npoints / 3))
            x = np.concatenate([x0, x1, x2])

            def function(x):
                return x - 0.1 * x ** 2 + np.cos(np.pi * x / 2)

            y = function(x)

            homo_noise_std = 0.25
            homo_noise = randn(*x.shape) * homo_noise_std
            y_homo = y + homo_noise

            hetero_noise_std = np.abs(0.1 * np.abs(x) ** 1.5)
            hetero_noise = randn(*x.shape) * hetero_noise_std
            y_hetero = y + hetero_noise

            X = x[:, np.newaxis]
            y_joint = np.stack([y_homo, y_hetero], axis=1)

            X_train, X_test, y_joint_train, y_joint_test = train_test_split(X, y_joint, test_size=0.5, random_state=42)
            y_hetero_train, y_hetero_test = y_joint_train[:, 1, np.newaxis], y_joint_test[:, 1, np.newaxis]
            y_homo_train, y_homo_test = y_joint_train[:, 0, np.newaxis], y_joint_test[:, 0, np.newaxis]

            x_means, x_stds = X_train.mean(axis=0), X_train.std(axis=0)
            y_hetero_means, y_hetero_stds = y_hetero_train.mean(axis=0), y_hetero_train.std(axis=0)
            y_homo_means, y_homo_stds = y_homo_test.mean(axis=0), y_homo_test.std(axis=0)

            X_train = ((X_train - x_means) / x_stds).astype(np.float32)
            X_test = ((X_test - x_means) / x_stds).astype(np.float32)

            y_hetero_train = ((y_hetero_train - y_hetero_means) / y_hetero_stds).astype(np.float32)
            y_hetero_test = ((y_hetero_test - y_hetero_means) / y_hetero_stds).astype(np.float32)

            y_homo_train = ((y_homo_train - y_homo_means) / y_homo_stds).astype(np.float32)
            y_homo_test = ((y_homo_test - y_homo_means) / y_homo_stds).astype(np.float32)

            if hetero:
                return X_train, y_hetero_train, X_test, y_hetero_test
            else:
                return X_train, y_homo_train, X_test, y_homo_test

        X_train, y_homo_train, X_test, y_homo_test = gen_my_1d()
        xy = np.concatenate([X_train, y_homo_train, X_test, y_homo_test], axis=1)
        np.save(base_dir + '/my_1d_data/my_1d_data.npy', xy)
        return X_train, y_homo_train, X_test, y_homo_test

    xy = np.load(base_dir + '/my_1d_data/my_1d_data.npy')
    X_train = xy[:, 0, None].astype(np.float32)
    y_homo_train = xy[:, 1, None].astype(np.float32)
    X_test = xy[:, 2, None].astype(np.float32)
    y_homo_test = xy[:, 3, None].astype(np.float32)

    return X_train, y_homo_train, X_test, y_homo_test

In [30]:
X_train, y_train, X_test, y_test = load_my_1d("./dataset")

In [32]:
print(len(X_train), len(y_train), len(X_test), len(y_test))

501 501 501 501


In [3]:
import pytorch_lightning as pl

In [4]:
train_dataset = dset.MNIST(root='./data', 
                                           train=True,
                                           transform=transforms.ToTensor(), 
                                           download=True)
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=256,
                          shuffle=True)

/home/me/.local/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [25]:
from cnfModel import CNFModel
learning_rate= 1e-3
batch_size = 256
model = CNFModel(learning_rate, batch_size)
trainer = pl.Trainer()
trainer.fit(model, train_loader, train_loader)
trainer.validate(model)

train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True,
                                           transform=transforms.ToTensor(), 
                                           download=False)
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size,
                          shuffle=True)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type           | Params
----------------------------------------
0 | flow | SequentialFlow | 1.8 K 
----------------------------------------
1.8 K     Trainable params
0         Non-trainable params
1.8 K     Total params
0.007     Total estimated model params size (MB)


Training T : 1.0
Number of trainable parameters of Point CNF: 1835


TypeError: forward() takes 2 positional arguments but 4 were given

In [23]:
x= torch.tensor([[1, 2, 3], [4, 5, 6]])
sqrt_end_time = torch.tensor(10)

y = torch.stack([torch.tensor(0.0).to(x), sqrt_end_time * sqrt_end_time]).to(x)
y

tensor([  0, 100])